# Identifying the causal variables in the Castle doctrine dataset

*Andrei Blahovici | Milena Kapralova | Luca Pantea | Paulius Skaisgiris*

This project is part of the Causality course at the UvA during fall 2023. We work with the Castle doctrine dataset (Cheng et al., 2012) to identify whether the castle (stand-your-ground) doctrine influences the rates of violence, and what the important variables influencing this relationship are. You can also find the dataset in [this repository](https://github.com/NickCH-K/causaldata/tree/main/Python/causaldata/castle).

### Imports

In [ ]:
# WARNING:
# The installation takes a few minutes.
# Only run during the first time running this notebook and if you don't have these packages installed.
# Run in terminal command line instead if it does not work.

# !pip install hyppo
# !pip install pingouin
# !pip install conditional_independence
!pip install ipywidgets

In [6]:
import numpy as np
import pandas as pd
from itertools import permutations
import conditional_independence
import hyppo
import matplotlib.pyplot as plt
import networkx as nx
import pingouin as pg
from sklearn import svm
from IPython.display import Image, display
from sklearn.linear_model import LinearRegression

# DoWhy
import dowhy
import dowhy.datasets
from dowhy import CausalModel
# from dowhy.causal_identifier import backdoor

# Hide some warnings
import warnings
from sklearn.exceptions import DataConversionWarning

# Seed
np.random.seed(42)

# Configs
plt.rcParams['figure.figsize'] = [5, 5]

### 1 Introduction and Motivation

Introduce the datasets, the assumptions and the causal questions you are investigating.

• Describe your dataset (e.g. what are the observational data and how they were collected, in case there are interventional data, also what are they and how they were collected).

• Describe the causal questions you wish to answer (e.g. “we investigate the effect of X on Y”).

• Describe the assumptions of your dataset (causal sufficiency, no cycles in the causal graph, positivity, etc).




False positive use of lethal force causes a net increase in homicides relative to the counterfactual.

<!-- <img src="us.png" width="200" height="500"> -->

<img src=https://d3i71xaburhd42.cloudfront.net/766441c1ab7f4390a5a8c0fa05ec9dc3cd4854d1/58-Figure1-1.png 
     align="center" 
     width="500" />
     
*Source: Shields (2016)*


**Dataset**

We use the FBI Uniform Crime Reports Summary Part I files from 2000 to 2010. The FBI Uniform Crime Reports is a harmonized data set on eight “index” crimes collected from voluntarily participating police agencies across the country. 

### 2 Exploratory Data Analysis

As shown in Tutorial 2. 

• Testing correlation / dependence for the variables in the dataset and show how they are dependent.

• Discuss the true causal graph of the dataset, if it’s known, and otherwise discuss a reasonable guess.

### 3 Identifying Estimands

As shown in Tutorials 3 and 4. Identify possible adjustment sets by hand by using:

• Backdoor criterion (most important)

• Frontdoor criterion

• Instrumental variables

Report what happens for these methods even if they don’t apply and explain why. Also show the results you get for each of these estimands from doWhy and compare with the ones you found by hand.

#### Backdoor Criterion (by hand)

In [ ]:
from itertools import chain, combinations

class BackdoorManager:
    '''
    This class takes care of the backdoor adjustment.
    
    :param G: a DiGraph object
    :param node_x: a node whose effect we are trying to predict
    :param node_y: a node effect on which we are trying to predict
    '''
    def __init__(self, G, node_x, node_y):
        self.G = G
        self.node_x = node_x
        self.node_y = node_y
        self.descendants_node_x = nx.descendants(self.G, node_x) | {node_x}
        
    def draw(self, pos, edge_color='black'):
        '''
        Draws the graph given a certain position and color of the nodes.
        '''
        nx.draw(self.G, pos=pos, with_labels=True, node_size=500, node_color='w', edgecolors='black', edge_color=edge_color)
        
    def write_gml(self, fname='backdoor_criterion_graph.gml'):
        nx.write_gml(G, fname)
        
    def get_all_paths(self):
        H = self.G.to_undirected()
        all_paths = list(nx.all_simple_paths(H, self.node_x, self.node_y))
        return all_paths
    
    def get_backdoor_paths(self):
        bd = backdoor.Backdoor(self.G, self.node_x, self.node_y)
        all_paths = self.get_all_paths()
        backdoor_paths = [path for path in all_paths if bd.is_backdoor(path)]
        return backdoor_paths
        
    def give_coll_noncoll_on_(self, path):
        '''
        Finds all colliders and non-colliders on a path.
        '''
        colliders = np.array([])
        non_colliders = []
        path_len = len(path)
        
        # Collider
        ## Loop through adjacent variables on the path, ignore the source and target variables as potential colliders
        for node0, node1, node2 in zip(path[0:path_len-2], path[1:path_len-1], path[2:]):
            if self.G.has_edge(node0, node1) and self.G.has_edge(node2, node1):
                ## Add the collider (and all its descendants) to the list
                colliders = np.append(colliders, list(nx.descendants(self.G,node1)) + [node1])
        colliders = colliders.flatten()
        
        # Non-collider
        non_colliders = [x for x in path[1:-1] if x not in colliders]

        return colliders, non_colliders
    
    def find_adjustment_variables(self):
        '''
        Performs the backdoor criterion search.
        '''
        self.adjustment_variables = pd.DataFrame(columns=['path', 'colliders', 'non_colliders'])
        paths = self.get_backdoor_paths()
        
        for path in paths:
            colliders, non_colliders = self.give_coll_noncoll_on_(path)
            self.adjustment_variables.loc[len(self.adjustment_variables.index)] = [path, colliders, non_colliders]
    
    def find_adjustment_sets(self, method='default'):
        '''
        Finds backdoor adjustment sets based on the adjustment variables and method. 
        Default method finds all the minimum-sized and maximum-sized adjustment sets, 
        
        :param method_name: {'default', 'exhaustive-search', 'minimal-adjustment', 'maximal-adjustment', 
                             'efficient-adjustment', 'efficient-minimal-adjustment', 'efficient-mincost-adjustment'}
        '''
        self.find_adjustment_variables()
        candidate_vars = set()
        for index, row in self.adjustment_variables.iterrows():
            candidate_vars.update(row['colliders'])
            candidate_vars.update(row['non_colliders'])
        candidate_vars.remove(self.node_x)
        candidate_vars.remove(self.node_y)
        
        all_combinations = list(chain.from_iterable(combinations(candidate_vars, r) for r in range(len(candidate_vars)+1)))
        
        # Checking which of the combinations are valid backdoor adjustment sets
        self.adjustment_sets = []
        for candidate_combination in all_combinations:
            valid = True
            candidate_combination = set(candidate_combination)
            for index, row in self.adjustment_variables.iterrows():
                cond1 = len(set(row['colliders']).intersection(candidate_combination)) == 0
                cond2 = len(set(row['non_colliders']).intersection(candidate_combination)) > 0 
#                 cond3 = len(set(row['non_colliders']).intersection(candidate_combination))
                
                
                if not (cond1 or cond2):
                    valid = False
                    break
            if valid:
                self.adjustment_sets.append(candidate_combination)
                
        print(len(self.adjustment_sets))

#### Backdoor Criterion (DoWhy)

In [ ]:
class BackdoorManagerDoWhy:
    '''
    This class takes care of the backdoor adjustment using DoWhy.
    
    :param G: a DiGraph object
    :param node_x: a node whose effect we are trying to predict
    :param node_y: a node effect on which we are trying to predict
    '''
    def __init__(self, fname='backdoor_criterion_graph.gml'):
        self.create_data()
        self.gml_to_string(fname)
        self.model = CausalModel(data = self.data,
                                 treatment='X',
                                 outcome='Y',
                                 graph=self.graph
                                )
    
    def create_data(self):
        self.data = pd.DataFrame({'A':[1],'B':[1],'C':[1],'D':[1],'W':[1],'X':[1], 'Y': [1], 'Z': [1]})
        
    def gml_to_string(self, file):
        gml_str = ''
        with open(file, 'r') as file:
            for line in file:
                gml_str += line.rstrip()
        self.graph = gml_str

    def draw(self):
        self.model.view_model()
        
    def find_adjustment_sets(self, method_name='default'): 
        '''
        Finds backdoor adjustment sets based on the method. 
        Default method finds all the minimum-sized and maximum-sized adjustment sets, 
        see https://github.com/py-why/dowhy/blob/main/dowhy/causal_model.py
        
        :param method_name: {'default', 'exhaustive-search', 'minimal-adjustment', 'maximal-adjustment', 
                             'efficient-adjustment', 'efficient-minimal-adjustment', 'efficient-mincost-adjustment'}
        '''
        identified_estimand = self.model.identify_effect(method_name=method_name)
        identifier = self.model.identifier
        adjustment_sets = identifier.identify_backdoor(self.model._graph, self.model._treatment, self.model._outcome)
        self.adjustment_sets = [back_set['backdoor_set'] for back_set in adjustment_sets]

### 4 Estimating Causal Effects

As shown in Tutorial 4. Apply and explain different causal estimate methods (linear, inverse propensity weighting, two-stage linear regression, etc.) to the previously identified estimands.

### 5 Causal Discovery

As shown in Tutorials 5 and 6. Try out the two types of algorithms for learning
causal graphs (constraint-based 10 % and score-based 10%). Explain why each method works or doesn’t and what is identifiable in terms of the causal graph.

• Run a constraint-based algorithm (e.g. PC) and a score-based algorithm (e.g. GES) on your data, and report back any identifiable causal relations.

• Optional: If you cannot find any identifiable causal relation or just want to test the algorithms further, simulate some data that resemble your real data (but maybe with less edges).

### 6 Validation and Sensitivity Analysis

Try out different ways to validate the results and do sensitivity analysis of the methods. 

• Report using some of the results of the refutation strategies implemented in DoWhy and interpret what they mean.

• Optional: If your dataset includes interventional data, check that the estimated causal effects from the observational data are reflected in the interventional data.

• Optional: Try experimenting with graphs in which some of the edges are dropped, and see how the results in Section 3 and 4 change.

• Optional: Try relaxing some of the assumptions you discussed in the Introduction, e.g. try to see the effect on not observing a certain variable

### 7 Discussion and Conclusion

In this part you will discuss the results of the previous sections and explain if they do answer the causal questions you described in the Introduction. You can also elaborate on the results you observed in the validation and discuss if the assumptions you had made initially were realistic.

### References

[1] Gretton, A., Fukumizu, K., Teo C., Song L., Schölkopf, B, and Alex Smola. A Kernel Statistical Test of Independence. Advances in Neural Information Processing Systems, 2007. https://proceedings.neurips.cc/paper/2007/file/d5cfead94f5350c12c322b5b664544c1-Paper.pdf

[2] Shields, J.C. (2016). Self-defense in the United States: A review of the literature.

### To-Do:

* Adjust arbitrary data in handcrafter dowhy for backdoor criterion
